In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import graphviz 

In [ ]:
data = pd.read_csv("tabela_final_treinamento.csv")

X = data.iloc[:, 0:225]
Y = data.iloc[:, -1]

In [ ]:
X

In [ ]:
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
classfier = tree.DecisionTreeClassifier()
classfier = classfier.fit(X_train, y_train)

In [ ]:
classfier.predict(X_test)

In [ ]:
dot_data = tree.export_graphviz(classfier, out_file=None, 
                      feature_names=data.columns.values.tolist()[:225],  
                      class_names=data.columns.values.tolist()[-1],  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  